In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
# Load the dataset
df = pd.read_csv("./base_ifood_limpa.csv", sep=";", header=0).drop('Unnamed: 0', axis=1)
categoricas = df.select_dtypes(include=['object']).columns.tolist()
numericas = df.select_dtypes(include=['number']).columns.tolist()
# Filtrar categorias específicas
categorias_selecionadas = ['Lanches', 'Brasileira', 'Pizza', 'Açaí']
df_filtrado = df[df['category'].isin(categorias_selecionadas)]
# Agrupar por categoria e calcular média do valor mínimo de pedido
df_filtrado.groupby('category')['minimumOrderValue'].agg(["mean"]).plot.bar(title="Média do valor mínimo de pedido por categoria")
plt.xlabel("Categoria")
df_filtrado.groupby('price_range')[["delivery_fee", "delivery_time"]].agg(["mean"])
df_filtrado.groupby('price_range')["delivery_fee"].agg(["mean"]).plot.bar(title="Média da taxa de entrega por faixa de preço")
plt.xlabel("Faixa de Preço")
plt.show()
df_filtrado.boxplot(column="minimumOrderValue", by="category").set_title("Distribuição do valor mínimo de pedido por categoria")
plt.suptitle("")

FileNotFoundError: [Errno 2] No such file or directory: './base_ifood_limpa.csv'

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Parâmetros do cenário
p_base = 0.10   # Conversão baseline
p_new = 0.06    # Conversão esperada da nova funcionalidade
revenue_per_conversion = 1500  # Receita por conversão (R$)
# Faixa de tamanhos de amostra
n_list = [100, 200, 300, 500, 1000, 2000, 5000]
# Número de simulações para cada n
num_simulations = 1000

# Listas para armazenar resultados
results = {
    'n': [],
    'avg_standard_error': [],
    'avg_loss': []
}
np.random.seed(42)  # Reprodutibilidade

for n in n_list:
    se_list = []
    loss_list = []
    
    for _ in range(num_simulations):
        # Gera dados de conversão (Bernoulli com p = p_new)
        conversions = np.random.binomial(1, p_new, n)
        
        # Taxa de conversão observada na amostra
        p_hat = np.mean(conversions)
        
        # Erro padrão (aprox.) da taxa de conversão
        se = np.std(conversions)/np.sqrt(n)
        
        se_list.append(se)
        
        # Perda de receita na amostra:
        # baseline = n * p_base * revenue_per_conversion
        # new = n * p_hat * revenue_per_conversion
        # loss = baseline - new
        loss = (p_base - p_hat) * n * revenue_per_conversion
        loss_list.append(loss)
    
    # Calcula as médias ao longo das simulações
    avg_se = np.mean(se_list)
    avg_loss = np.mean(loss_list)
    
    # Armazena nos resultados
    results['n'].append(n)
    results['avg_standard_error'].append(avg_se)
    results['avg_loss'].append(avg_loss)
# Converte resultados em DataFrame
df_results = pd.DataFrame(results)
print(df_results)
# Visualização
fig, ax1 = plt.subplots(figsize=(8, 5))

color1 = 'tab:blue'
ax1.set_xlabel('Tamanho da Amostra (n)')
ax1.set_ylabel('Erro Padrão Médio', color=color1)
ax1.plot(df_results['n'], df_results['avg_standard_error'], marker='o', color=color1, label='Erro Padrão')
ax1.tick_params(axis='y', labelcolor=color1)

# Cria um segundo eixo Y para a perda de receita
ax2 = ax1.twinx()  
color2 = 'tab:red'
ax2.set_ylabel('Perda Média de Receita (R$)', color=color2)
ax2.plot(df_results['n'], df_results['avg_loss'], marker='s', color=color2, label='Perda de Receita')
ax2.tick_params(axis='y', labelcolor=color2)

plt.title('Trade-off: Erro Padrão vs. Risco de Perda de Receita\nNova Funcionalidade')
fig.tight_layout()
plt.show()